## Questão 1


In [1]:

import numpy as np
import os
import soundfile as sf
from scipy.signal import lfilter

# =====================================================
# 1. FUNÇÕES AUXILIARES
# =====================================================

def apply_delay(x, m):
    """
    Aplica um atraso puro de m amostras ao sinal x
    """
    y = np.zeros(len(x) + m)
    y[m:] = x
    return y


def schroeder_reverb(x, Fs, RT60):
    """
    Reverberador de Schroeder (TP1)
    Implementa o campo reverberante R(z)
    """
    # RT60 = 0 -> espaço livre (sem reverberação)
    if RT60 == 0:
        return np.zeros_like(x)

    # Atrasos típicos do reverberador
    comb_delays = np.array([1557, 1617, 1491, 1422])
    ap_delays   = np.array([225, 556])

    # Ganhos dos filtros comb em função do RT60
    g_comb = 10 ** (-3 * comb_delays / (RT60 * Fs))

    # Banco de filtros comb em paralelo
    y = np.zeros_like(x, dtype=float)
    for d, g in zip(comb_delays, g_comb):
        b = np.zeros(d + 1)
        b[0]  = 1
        b[-1] = -g
        a = np.zeros(d + 1)
        a[0] = 1
        y += lfilter(b, a, x)

    # Filtros all-pass em cascata
    g_ap = 0.7
    for d in ap_delays:
        b = np.zeros(d + 1)
        a = np.zeros(d + 1)
        b[0]  = -g_ap
        b[-1] = 1
        a[0]  = 1
        a[-1] = g_ap
        y = lfilter(b, a, y)

    return y


# =====================================================
# 2. PARÂMETROS FÍSICOS E ACÚSTICOS
# =====================================================

Fs = 48000          # frequência de amostragem [Hz]
c  = 343            # velocidade do som [m/s]

# Distâncias da fonte ao ouvinte (exemplo)
d1 = 1.5            # campo direto
d2 = 2.2            # reflexão 1 (biombo)
d3 = 3.1            # reflexão 2 (biombo)

# Conversão distância -> atraso (amostras)
m1 = int(d1 / c * Fs)
m2 = int(d2 / c * Fs)
m3 = int(d3 / c * Fs)

# Atenuação geométrica
a1 = 1 / d1
a2 = 1 / d2
a3 = 1 / d3


# =====================================================
# 3. LEITURA DO SINAL DE ENTRADA
# =====================================================

folder = "FragileThoughts/"
tracks = [f for f in os.listdir(folder) if f.lower().endswith(".wav")]

# Usa apenas uma pista para a Questão 1
x, Fs = sf.read(folder + tracks[0])

# Garantir sinal mono
if x.ndim > 1:
    x = x[:, 0]

print("A processar:", tracks[0])


# =====================================================
# QUESTÃO 1 (a)
# CAMPO DIRETO
# =====================================================
# Modelo: a1 * x[n - m1]

campo_direto = a1 * apply_delay(x, m1)
sf.write("Q1_a_campo_direto.wav", campo_direto, Fs)


# =====================================================
# QUESTÃO 1 (b)
# PRIMEIRAS REFLEXÕES (ORDEM 1)
# =====================================================
# Cada reflexão é modelada como um atraso e atenuação
# independentes (modelo FIR)

reflexao_1 = a2 * apply_delay(x, m2)
reflexao_2 = a3 * apply_delay(x, m3)

# Igualar comprimentos
L = max(len(reflexao_1), len(reflexao_2))
reflexao_1 = np.pad(reflexao_1, (0, L - len(reflexao_1)))
reflexao_2 = np.pad(reflexao_2, (0, L - len(reflexao_2)))

campo_reflexoes = reflexao_1 + reflexao_2
sf.write("Q1_b_primeiras_reflexoes.wav", campo_reflexoes, Fs)


# =====================================================
# QUESTÃO 1 (c)
# CAMPO REVERBERANTE (R(z))
# =====================================================
# Reverberação da sala para vários valores de RT60

RT60_values = [0, 0.5, 2, 10]

for RT in RT60_values:
    campo_reverberante = schroeder_reverb(x, Fs, RT)
    sf.write(f"Q1_c_campo_reverberante_RT{RT}.wav",
             campo_reverberante, Fs)


# =====================================================
# QUESTÃO 1 (d)
# CAMPO TOTAL
# Campo direto + reflexões + reverberação
# =====================================================

for RT in RT60_values:

    campo_reverberante = schroeder_reverb(x, Fs, RT)

    # Igualar comprimentos de todos os sinais
    L = max(len(campo_direto),
            len(campo_reflexoes),
            len(campo_reverberante))

    cd = np.pad(campo_direto, (0, L - len(campo_direto)))
    cr = np.pad(campo_reflexoes, (0, L - len(campo_reflexoes)))
    rv = np.pad(campo_reverberante, (0, L - len(campo_reverberante)))

    campo_total = cd + cr + rv

    sf.write(f"Q1_d_campo_total_RT{RT}.wav",
             campo_total, Fs)


print("Questão 1 concluída com sucesso.")


A processar: 01_Kick.wav
Questão 1 concluída com sucesso.


## Questão 2

In [2]:


import numpy as np
import soundfile as sf
from scipy.signal import lfilter

# =====================================================
# 1. FUNÇÕES AUXILIARES
# =====================================================

def apply_delay(x, m):
    y = np.zeros(len(x) + m)
    y[m:] = x
    return y


def schroeder_reverb(x, Fs, RT60):
    if RT60 == 0:
        return np.zeros_like(x)

    comb_delays = np.array([1557, 1617, 1491, 1422])
    ap_delays   = np.array([225, 556])
    g_comb = 10 ** (-3 * comb_delays / (RT60 * Fs))

    y = np.zeros_like(x, dtype=float)
    for d, g in zip(comb_delays, g_comb):
        b = np.zeros(d + 1)
        b[0], b[-1] = 1, -g
        a = np.zeros(d + 1)
        a[0] = 1
        y += lfilter(b, a, x)

    g_ap = 0.7
    for d in ap_delays:
        b = np.zeros(d + 1)
        a = np.zeros(d + 1)
        b[0], b[-1] = -g_ap, 1
        a[0], a[-1] = 1, g_ap
        y = lfilter(b, a, y)

    return y


def binaural_hrtf(x, Fs, angle_deg):
    """
    HRTF simplificada (plano horizontal)
    Usa ITD + ILD
    """
    c = 343
    head_radius = 0.0875  # raio médio da cabeça [m]
    angle = np.deg2rad(angle_deg)

    # ITD (modelo de Woodworth)
    itd = (head_radius / c) * (angle + np.sin(angle))
    itd_samples = int(abs(itd) * Fs)

    # ILD simples
    ild = 0.7 + 0.3 * np.cos(angle)

    if angle >= 0:  # fonte à direita
        xL = apply_delay(x, itd_samples) * ild
        xR = x
    else:           # fonte à esquerda
        xL = x
        xR = apply_delay(x, itd_samples) * ild

    L = max(len(xL), len(xR))
    xL = np.pad(xL, (0, L - len(xL)))
    xR = np.pad(xR, (0, L - len(xR)))

    return xL, xR


# =====================================================
# 2. PARÂMETROS
# =====================================================

Fs = 48000
c  = 343

# distâncias
d1, d2, d3 = 1.5, 2.2, 3.1
m1 = int(d1 / c * Fs)
m2 = int(d2 / c * Fs)
m3 = int(d3 / c * Fs)

# ângulos das fontes/reflexões
ang_direto = 30     # graus
ang_ref1   = -60
ang_ref2   = 110


# =====================================================
# 3. SINAL DE ENTRADA
# =====================================================

x, Fs = sf.read("FragileThoughts/" + os.listdir("FragileThoughts/")[0])
if x.ndim > 1:
    x = x[:, 0]


# =====================================================
# QUESTÃO 2 (a)
# Coeficiente de absorção α
# =====================================================

alphas = [0, 0.5, 0.8, 1]

for alpha in alphas:

    # Atenuações com absorção
    a1 = 1 / d1
    a2 = (1 - alpha) / d2
    a3 = (1 - alpha) / d3

    # Campo direto binaural
    cd = a1 * apply_delay(x, m1)
    cd_L, cd_R = binaural_hrtf(cd, Fs, ang_direto)

    # Reflexões binaurais
    r1 = a2 * apply_delay(x, m2)
    r2 = a3 * apply_delay(x, m3)

    r1_L, r1_R = binaural_hrtf(r1, Fs, ang_ref1)
    r2_L, r2_R = binaural_hrtf(r2, Fs, ang_ref2)

    # Soma binaural
    L = max(len(cd_L), len(r1_L), len(r2_L))
    left  = np.pad(cd_L, (0, L-len(cd_L))) \
          + np.pad(r1_L, (0, L-len(r1_L))) \
          + np.pad(r2_L, (0, L-len(r2_L)))

    right = np.pad(cd_R, (0, L-len(cd_R))) \
          + np.pad(r1_R, (0, L-len(r1_R))) \
          + np.pad(r2_R, (0, L-len(r2_R)))

    sf.write(f"Q2a_binaural_alpha{alpha}.wav",
             np.column_stack((left, right)), Fs)


# =====================================================
# QUESTÃO 2 (b)
# Campo reverberante + binaural
# =====================================================

RT60_vals = [0, 0.5, 2, 10]

for alpha in alphas:
    for RT in RT60_vals:

        # reutiliza campo total mono
        campo_mono = left + right
        campo_rev  = schroeder_reverb(campo_mono, Fs, RT)

        rev_L, rev_R = binaural_hrtf(campo_rev, Fs, 0)

        out_L = left + rev_L
        out_R = right + rev_R

        sf.write(f"Q2b_binaural_alpha{alpha}_RT{RT}.wav",
                 np.column_stack((out_L, out_R)), Fs)

print("Questão 2 concluída.")


Questão 2 concluída.


## Questão 3

## Questão 4